# Item 51: Prefer Class Decorators Over Metaclasses for Composable Class Extensions

Although metaclasses allow us to customize class creation in multiple ways, they still fall short of handling every situation that may arise.

In [2]:
# Here we define the debugging Decorator
from functools import wraps

def trace_func(func):
    if hasattr(func, 'tracing'): # Only decorate once
        return func

    @wraps(func)
    def wrapper(*args, **kwargs):
        result = None
        try:
            result = func(*args, **kwargs)
            return result
        except Exception as e:
            result = e
            raise
        finally:
            print(f'{func.__name__} ({args!r}, {kwargs!r}) -> {result!r}')

    wrapper.tracing = True
    return wrapper


In [3]:
# We can apply this decorator to varios special methods in our new dict subclass
class TraceDict(dict):
    @trace_func
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @trace_func
    def __setitem__(self, *args, **kwargs):
        return super().__setitem__(*args, **kwargs)

    @trace_func
    def __getitem__(self, *args, **kwargs):
        return super().__getitem__(*args, **kwargs)

    # ...


In [5]:
# We can verify that these methods are decorated by interacting with an instance of the class
trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']
try:
    trace_dict['does not exist']
except:
    pass # Expected

__init__ (({'hi': 1}, [('hi', 1)]), {}) -> None
__setitem__ (({'hi': 1, 'there': 2}, 'there', 2), {}) -> None
__getitem__ (({'hi': 1, 'there': 2}, 'hi'), {}) -> 1
__getitem__ (({'hi': 1, 'there': 2}, 'does not exist'), {}) -> KeyError('does not exist')


The problem with this code is that we had to redefine all of the methods that we wanted to decorate with `@trace_func`. This is redundant boiler that's hard to read and error prone. Further, if a new method is later added to the `dict` superclass, it won't be decorated unless we alse define it in `TraceDict`.

One way to solve this problem is to use a metaclass to automatically decorate all methods of a class.

In [11]:
import types

trace_types = (
    types.MethodType,
    types.FunctionType,
    types.BuiltinFunctionType,
    types.BuiltinMethodType,
    types.MethodDescriptorType,
    types.ClassMethodDescriptorType
)

class TraceMeta(type):
    def __new__(meta, name, bases, class_dict):
        klass = super().__new__(meta, name, bases, class_dict)

        for key in dir(klass):
            value = getattr(klass, key)
            if isinstance(value, trace_types):
                wrapped = trace_func(value)
                setattr(klass, key, wrapped)

        return klass

In [12]:
# Now, we can declare my dict subclass by using the TraceMeta metaclass and verify that it works
class TraceDict(dict, metaclass=TraceMeta):
    pass

trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']
try:
    trace_dict['does not exist']
except KeyError:
    pass  # Expected


__new__ ((<class '__main__.TraceDict'>, [('hi', 1)]), {}) -> {}
__getitem__ (({'hi': 1, 'there': 2}, 'hi'), {}) -> 1
__getitem__ (({'hi': 1, 'there': 2}, 'does not exist'), {}) -> KeyError('does not exist')


In [13]:
# What happens if we try to use TraceMeta when a superclass already has specified a metaclass?
class OtherMeta(type):
    pass

class SimpleDict(dict, metaclass=OtherMeta):
    pass

class TraceDict(SimpleDict, metaclass=TraceMeta):
    pass

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [14]:
# This fails because TraceMeta does not inherit from OtherMeta. In theory, we can use metaclass inheritance 
# to solve this problem by having OtherMeta inherit from TraceMeta
class TraceMeta(type):
    def __new__(meta, name, bases, class_dict):
        klass = super().__new__(meta, name, bases, class_dict)

        for key in dir(klass):
            value = getattr(klass, key)
            if isinstance(value, trace_types):
                wrapped = trace_func(value)
                setattr(klass, key, wrapped)

        return klass

class OtherMeta(TraceMeta):
    pass


class SimpleDict(dict, metaclass=OtherMeta):
    pass


class TraceDict(SimpleDict, metaclass=TraceMeta):
    pass


trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']
try:
    trace_dict['does not exist']
except KeyError:
    pass  # Expected


__init_subclass__ ((), {}) -> None
__new__ ((<class '__main__.TraceDict'>, [('hi', 1)]), {}) -> {}
__getitem__ (({'hi': 1, 'there': 2}, 'hi'), {}) -> 1
__getitem__ (({'hi': 1, 'there': 2}, 'does not exist'), {}) -> KeyError('does not exist')


But, the above won't work if the metaclass is from a library that we can't modify, or if we want to use multiple utility classes like `TraceMeta` at the same time. The metaclass approach puts too many constraints on the class that's being modified.

To solve this, Python supports *class decorators*. Class decorators work just like function decorators: They're applied with the `@` symbol prefixing a function before the class declaration. The function is expected to modify or re-create the class accordingly and then return it:

In [15]:
def my_class_decorator(klass):
    klass.extra_param = 'hello'
    return klass

@my_class_decorator
class MyClass:
    pass

print(MyClass)
print(MyClass.extra_param)

<class '__main__.MyClass'>
hello


In [16]:
# We can implement a class decorator to apply trace_func to all methods and functions of a class by moving
# the core of the TraceMeta.__new__ method above into a stand-alone fucntion. This implementation is much 
# shorter that the metaclass version
def trace(klass):
    for key in dir(klass):
        value = getattr(klass, key)
        if isinstance(value, trace_types):
            wrapped = trace_func(value)
            setattr(klass, key, wrapped)
    return klass

In [17]:
# We can apply this decorator to our dict subclass to get the same behavior as we get by using the metaclass 
# approach above
@trace
class TraceDict(dict):
    pass


trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']
try:
    trace_dict['does not exist']
except KeyError:
    pass  # Expected


__new__ ((<class '__main__.TraceDict'>, [('hi', 1)]), {}) -> {}
__getitem__ (({'hi': 1, 'there': 2}, 'hi'), {}) -> 1
__getitem__ (({'hi': 1, 'there': 2}, 'does not exist'), {}) -> KeyError('does not exist')


In [18]:
# Class decorators also work when the class being decorated already has a metaclass
class OtherMeta(type):
    pass

@trace
class TraceDict(dict, metaclass=OtherMeta):
    pass


trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']
try:
    trace_dict['does not exist']
except KeyError:
    pass  # Expected


__new__ ((<class '__main__.TraceDict'>, [('hi', 1)]), {}) -> {}
__getitem__ (({'hi': 1, 'there': 2}, 'hi'), {}) -> 1
__getitem__ (({'hi': 1, 'there': 2}, 'does not exist'), {}) -> KeyError('does not exist')


When we're looking for composable ways to extend classes, class decorators are the best tool for the job.